# Scraping Lead Posts

In [ ]:
# Import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from dateutil import relativedelta
import time
import requests
from bs4 import BeautifulSoup

In [ ]:
url_start = "https://www.letemps.ch/archive/"
data = []
missed_data = []
links = pd.read_excel("../databases/Pipeline2_keywords.xlsx")
links = links['Link'].to_list()

In [ ]:
# Scraping Lead Posts
for i in links:
    url = i
    headers = {"User-Agent": "Mozilla/5.0"}

    success = False
    retries = 0
    max_retries = 5 

    while not success and retries < max_retries:
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code == 429:
                print(f"Rate limited (429) on {url}, retrying...")
                time.sleep(1)
                retries += 1
                continue

            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            post_lead = soup.find(class_="post__lead")
            if post_lead:
                articles = post_lead.select("p")
                for article in articles:
                    lead_post = article.get_text(strip=True)

                    data.append({"Title": lead_post, "Link": i})
            else:
                print(f"No 'lead_post' section found on {url}")

            success = True  # Success: exit loop

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}, retrying...")
            time.sleep(1)
            retries += 1

    if not success:
        print(f"Failed after retries: {url}")
        missed_data.append(url)


In [ ]:
df = pd.DataFrame(data=data, columns= ["Title", "Link"])
missed_data = pd.DataFrame(data=missed_data, columns= ["Link"])

df.to_excel("databases/Pipeline2_toapply.xlsx",sheet_name='Le_Temps')  # Pipeline1_LeadPosts_Le_Temps.xlsx for Pipeline 1
#missed_data.to_excel("shelfens/Missed_data_LP2.xlsx", sheet_name="Titles")
